# 1. Instanciate URIs for terms (legislatura) 

# 3. Allocate Posts and Memberships

 Solves URI to Post as interval scheduling problem.

In [1]:
import json 
import pandas as pd
import sys
import re
from collections import deque

start_date = '2015-02-01'

# converts xml 2 dataframe
from models.xml2dataframe import XML2DataFrame

# reads XML files 
import xml.etree.ElementTree as ET 
tree = ET.parse('datasets/camara/ObterDeputados.xml') # api v1 this list is volatile and has 513 deputados
xmlstr_current_congressmen = ET.tostring(tree.getroot(), encoding='utf-8', method='xml')


In [2]:
with open('datasets/slp/posts/posts_camara.csv', mode='r') as f:
    postsstr = f.read()
f.close()
data = re.split(r'\n|;', postsstr)[3:]
posts = data[0:-1:3]
postsdeque_dict = {start_date:deque(posts, maxlen=len(posts))} 
print(len(posts))

520


In [3]:
with open('datasets/camara/json/memberships_with_role_deputado-55.json', mode='r') as f:
    membership_with_role_deputado = json.load(f)
f.close()
df = pd.DataFrame.from_dict(membership_with_role_deputado)
print(df.shape[0])
df.head()


953


,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri
0,2017-10-30,2015-02-01,Licença Conjunta Consecutiva,None,20,MT,Titular,ba586997-d2ed-4799-8f59-2a9c8d11f16b,b27beba7-ca02-4041-a9e0-1793bcd141fe,6b1651f1-f205-4840-b3d1-d64166fd2b51
1,None,2018-02-28,None,None,None,MT,Titular,ba586997-d2ed-4799-8f59-2a9c8d11f16b,b27beba7-ca02-4041-a9e0-1793bcd141fe,27cd6717-1dfc-425a-acb8-547e9fbaf7ba
2,2017-08-02,2015-04-09,Afastamento,185500,53,MA,Suplente,cac2beba-01c8-4c9a-a96f-4ed943b19515,b27beba7-ca02-4041-a9e0-1793bcd141fe,fc88e06c-3569-48a1-898d-69cea8b53813
3,2017-10-20,2017-08-03,Afastamento,74210,53,MA,Suplente,cac2beba-01c8-4c9a-a96f-4ed943b19515,b27beba7-ca02-4041-a9e0-1793bcd141fe,0e1d10d5-b34b-4a41-a126-87abbd54a14f
4,None,2017-10-26,None,74210,None,MA,Suplente,cac2beba-01c8-4c9a-a96f-4ed943b19515,b27beba7-ca02-4041-a9e0-1793bcd141fe,e7fcd62a-6e99-49f5-b61c-9d84db38f9eb


In [4]:
#Store as resource 
fields = ['org:hasMember', 'org:role', 'slp:resource_uri', 'cam:dataFim', 'cam:dataInicio']
mapper = {'cam:dataFim' :'time:finishDate', 'cam:dataInicio':'time:startDate'}
df_memberships = df[fields]
df_memberships = df_memberships.rename(mapper, axis = 'columns')
df_memberships.to_csv('datasets/slp/memberships/memberships_congress-55.csv', sep=';', index= None,encoding= 'utf-8')

df_memberships.head()

,org:hasMember,org:role,slp:resource_uri,time:finishDate,time:startDate
0,ba586997-d2ed-4799-8f59-2a9c8d11f16b,b27beba7-ca02-4041-a9e0-1793bcd141fe,6b1651f1-f205-4840-b3d1-d64166fd2b51,2017-10-30,2015-02-01
1,ba586997-d2ed-4799-8f59-2a9c8d11f16b,b27beba7-ca02-4041-a9e0-1793bcd141fe,27cd6717-1dfc-425a-acb8-547e9fbaf7ba,None,2018-02-28
2,cac2beba-01c8-4c9a-a96f-4ed943b19515,b27beba7-ca02-4041-a9e0-1793bcd141fe,fc88e06c-3569-48a1-898d-69cea8b53813,2017-08-02,2015-04-09
3,cac2beba-01c8-4c9a-a96f-4ed943b19515,b27beba7-ca02-4041-a9e0-1793bcd141fe,0e1d10d5-b34b-4a41-a126-87abbd54a14f,2017-10-20,2017-08-03
4,cac2beba-01c8-4c9a-a96f-4ed943b19515,b27beba7-ca02-4041-a9e0-1793bcd141fe,e7fcd62a-6e99-49f5-b61c-9d84db38f9eb,None,2017-10-26


In [5]:
def sortfinish(x):
    if not x['cam:dataFim']:
        return '2999-01-01'
    return x['cam:dataFim']

def sortstart(x):
    return x['cam:dataInicio']

def pop(process_date):
    key = max([k for k in postsdeque_dict 
               if ((k <= process_date) and (len(postsdeque_dict[k]) > 0))])
    this_post =  postsdeque_dict[key].pop()
    return this_post

def push(process_date, this_post):
    if process_date not in postsdeque_dict:
        postsdeque_dict[process_date] = deque([this_post] , maxlen=315)
    else:
        postsdeque_dict[process_date].append(this_post) 

def peeklen(process_date):
    keys = [k for k in postsdeque_dict 
               if ((k <= process_date) and (len(postsdeque_dict[k]) > 0))]
    return sum([len(postsdeque_dict[k]) for k in keys])

In [6]:
membership_with_role_deputado = sorted(
    sorted( membership_with_role_deputado, key=sortstart),
    key=sortfinish
)
df = pd.DataFrame.from_dict(membership_with_role_deputado)

# df.to_csv('datasets/memberships_with_role_deputado-55.csv', sep=';', encoding='utf-8')
df.head()

,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri
0,2015-02-02,2015-02-01,Ministro de Estado,None,11,SP,Titular,8056c06c-0b0b-4fce-852b-cbd4869c8e55,b27beba7-ca02-4041-a9e0-1793bcd141fe,13814ad8-d3a2-4f48-a106-8bb409b03ecd
1,2015-02-02,2015-02-01,Ministro de Estado,None,11,MG,Titular,aea74d14-4f81-4151-b968-daf588f4d92c,b27beba7-ca02-4041-a9e0-1793bcd141fe,e91acff0-dfa3-481a-9121-c66851cf4846
2,2015-02-02,2015-02-01,Ministro de Estado,None,11,MG,Titular,e914adad-d845-43ae-af4a-c9b4fb878778,b27beba7-ca02-4041-a9e0-1793bcd141fe,7426a324-8fc9-49e4-a63a-b23ffbd7619a
3,2015-02-02,2015-02-01,Ministro de Estado,None,11,RS,Titular,a0822a0a-d508-498b-830d-ede9419642ce,b27beba7-ca02-4041-a9e0-1793bcd141fe,1748aea9-f655-4de4-85d9-cd2c90c36495
4,2015-02-03,2015-02-01,Secretário de Estado,None,12,SP,Titular,8ed64c8f-ccb1-45cb-b1fe-ad0ad5dda500,b27beba7-ca02-4041-a9e0-1793bcd141fe,7007fc34-3f62-47c8-8917-343dbc403068


In [7]:

_message = '{:}:processing_date:{:}\tfinish_date:{:}\tnum_vacancies:{:}\tincoming:{:}\n'
with open('datasets/crole_deputado_with_posts-55.csv', mode='w+') as f:
    f.write('org:role;org:post\n')
    
    for i, d in enumerate(membership_with_role_deputado):
        processing_date = d['cam:dataInicio']
        finish_date = d['cam:dataFim']
        
        # pops from processing_date
        this_post = pop(processing_date)
        num_posts = peeklen(processing_date)
        incoming= peeklen('2999-01-01') - num_posts

        sys.stdout.write(_message.format(i + 1, processing_date, finish_date, num_posts, incoming))
        # pushes on finish_date
        if finish_date:     
            push(finish_date, this_post)
                
        buff = '{:};{:}\n'.format(d['org:role'] , this_post)
        f.write(buff)


1:processing_date:2015-02-01	finish_date:2015-02-02	num_vacancies:519	incoming:0
2:processing_date:2015-02-01	finish_date:2015-02-02	num_vacancies:518	incoming:1
3:processing_date:2015-02-01	finish_date:2015-02-02	num_vacancies:517	incoming:2
4:processing_date:2015-02-01	finish_date:2015-02-02	num_vacancies:516	incoming:3
5:processing_date:2015-02-01	finish_date:2015-02-03	num_vacancies:515	incoming:4
6:processing_date:2015-02-01	finish_date:2015-02-03	num_vacancies:514	incoming:5
7:processing_date:2015-02-01	finish_date:2015-02-03	num_vacancies:513	incoming:6
8:processing_date:2015-02-01	finish_date:2015-02-03	num_vacancies:512	incoming:7
9:processing_date:2015-02-01	finish_date:2015-02-03	num_vacancies:511	incoming:8
10:processing_date:2015-02-01	finish_date:2015-02-03	num_vacancies:510	incoming:9
11:processing_date:2015-02-01	finish_date:2015-02-03	num_vacancies:509	incoming:10
12:processing_date:2015-02-01	finish_date:2015-02-04	num_vacancies:508	incoming:11
13:processing_date:2015

 # Deputados em exercício

In [8]:
xml2df = XML2DataFrame(xmlstr_current_congressmen)
df_current_congressman = xml2df.process_data()
df_current_congressman.head()

,anexo,codOrcamento,comissoes,condicao,deputado,email,fone,gabinete,idParlamentar,ideCadastro,matricula,nome,nomeParlamentar,partido,sexo,uf,urlFoto
0,4,2804,\n,Suplente,\n,dep.junjiabe@camara.gov.br,3215-5512,512,1635214,160544,610,JUNJI ABE,JUNJI ABE,PSD,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...
1,4,3611,\n,Titular,\n,dep.luizaerundina@camara.gov.br,3215-5620,620,5830288,74784,371,LUIZA ERUNDINA DE SOUSA,LUIZA ERUNDINA,PSOL,feminino,SP,http://www.camara.gov.br/internet/deputado/ban...
2,4,2615,\n,Titular,\n,dep.robertoalves@camara.gov.br,3215-5946,946,5830289,152610,387,BENEDITO ROBERTO ALVES FERREIRA,ROBERTO ALVES,PRB,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...
3,3,2429,\n,Titular,\n,dep.lazarobotelho@camara.gov.br,3215-5478,478,5830290,141480,63,LÁZARO BOTELHO MARTINS,LÁZARO BOTELHO,PP,masculino,TO,http://www.camara.gov.br/internet/deputado/ban...
4,4,2471,\n,Titular,\n,dep.sergiobrito@camara.gov.br,3215-5638,638,5830291,73808,213,SÉRGIO LUÍS LACERDA BRITO,SÉRGIO BRITO,PSD,masculino,BA,http://www.camara.gov.br/internet/deputado/ban...


 # Deputados SLP

In [9]:
with open('datasets/camara2/json/congressmen-55.json', mode='r') as f:
    agentsstr = f.read()
f.close()
lst = json.loads(agentsstr)

df_agents = pd.DataFrame.from_dict(lst)
df_agents.head()

,cam2:cpf,cam2:dataFalecimento,cam2:dataNascimento,cam2:escolaridade,cam2:id,cam2:municipioNascimento,cam2:nomeCivil,cam2:redeSocial,cam2:sexo,cam2:ufNascimento,cam2:uri,slp:old_resource_uri,slp:resource_uri
0,,None,1962-04-25,None,136811,Hulha Negra,JOSÉ ALFONSO EBERT HAMM,[],M,RS,https://dadosabertos.camara.leg.br/api/v2/depu...,5de584aa861a023b8022f2d89ade4bc9,248c58a8-f41b-437b-9493-070734f3a9b3
1,,None,1976-10-23,Pós-Graduação,178836,Rio Branco,ALAN RICK MIRANDA,[],M,AC,https://dadosabertos.camara.leg.br/api/v2/depu...,3f8ce45065ead34564ffbb987a294be7,4f885f56-c35e-4314-b7fe-d4593f82d7fd
2,,None,1969-02-13,None,160527,Campina Grande,AGUINALDO VELLOSO BORGES RIBEIRO,[],M,PB,https://dadosabertos.camara.leg.br/api/v2/depu...,763cbe309ac7581476388a53524c117d,c7a9b0af-83e8-46c2-b652-820d86303dcc
3,,None,1956-02-05,Superior,178903,Nova Veneza,ADILTON DOMINGOS SACHETTI,[],M,SC,https://dadosabertos.camara.leg.br/api/v2/depu...,d2488aea4c792db37a71ef88be6c75f7,ba586997-d2ed-4799-8f59-2a9c8d11f16b
4,,None,1964-05-30,None,133374,Teófilo Otoni,ADEMIR CAMILO PRATES RODRIGUES,[],M,MG,https://dadosabertos.camara.leg.br/api/v2/depu...,0619a608d90f2633bc0b259580c3b13e,dac5dff3-c697-425a-9262-802606c14027


In [10]:
df.head()

,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri
0,2015-02-02,2015-02-01,Ministro de Estado,None,11,SP,Titular,8056c06c-0b0b-4fce-852b-cbd4869c8e55,b27beba7-ca02-4041-a9e0-1793bcd141fe,13814ad8-d3a2-4f48-a106-8bb409b03ecd
1,2015-02-02,2015-02-01,Ministro de Estado,None,11,MG,Titular,aea74d14-4f81-4151-b968-daf588f4d92c,b27beba7-ca02-4041-a9e0-1793bcd141fe,e91acff0-dfa3-481a-9121-c66851cf4846
2,2015-02-02,2015-02-01,Ministro de Estado,None,11,MG,Titular,e914adad-d845-43ae-af4a-c9b4fb878778,b27beba7-ca02-4041-a9e0-1793bcd141fe,7426a324-8fc9-49e4-a63a-b23ffbd7619a
3,2015-02-02,2015-02-01,Ministro de Estado,None,11,RS,Titular,a0822a0a-d508-498b-830d-ede9419642ce,b27beba7-ca02-4041-a9e0-1793bcd141fe,1748aea9-f655-4de4-85d9-cd2c90c36495
4,2015-02-03,2015-02-01,Secretário de Estado,None,12,SP,Titular,8ed64c8f-ccb1-45cb-b1fe-ad0ad5dda500,b27beba7-ca02-4041-a9e0-1793bcd141fe,7007fc34-3f62-47c8-8917-343dbc403068


In [25]:
df2 = df.merge(df_agents, left_on='org:hasMember', right_on='slp:resource_uri', how='inner')
print(df2.shape[0])
df2.head()

0


,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri_x,...,cam2:escolaridade,cam2:id,cam2:municipioNascimento,cam2:nomeCivil,cam2:redeSocial,cam2:sexo,cam2:ufNascimento,cam2:uri,slp:old_resource_uri,slp:resource_uri_y


In [11]:
# df2.to_csv('datasets/membership_with_role_deputado_and_foaf-55.csv', sep=';', encoding='utf-8')

In [13]:

index = df2['cam:dataFim'].isnull().values
current_from_memberships = set(df2.loc[index, 'cam2:nomeCivil'].values)
current_from_xml = set(df_current_congressman['nome'].values)

print(len(current_from_memberships), len(current_from_xml))
current_from_memberships - current_from_xml

513 513


{'ELIZEU DIONIZIO SOUZA  DA SILVA', 'JOAO HENRIQUE HOLANDA  CALDAS'}

In [14]:
current_from_xml - current_from_memberships

{'ELIZEU DIONIZIO SOUZA DA SILVA', 'JOAO HENRIQUE HOLANDA CALDAS'}